In [1]:
import pandas as pd
import numpy as np
import re
import random


SEED = 93
random.seed(SEED)
np.random.seed(SEED)

In [2]:
recipe_df = pd.read_csv("../data/recipes.csv", index_col=0)
recipe_df.head()

,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
2,Sarah's Homemade Applesauce,10 mins,15 mins,25 mins,4,NaN,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...","Prep Time: 10 mins, Cook Time: 15 mins, Total ...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...
3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,/Desserts/Crisps and Crumbles Recipes/Apple Cr...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...


In [3]:
for c in recipe_df.columns:
    if recipe_df[c].dtype != "object":
        continue
    recipe_df.loc[recipe_df[c].str.strip() == "", c] = np.NaN

recipe_df.isna().sum()

recipe_name       0
prep_time        51
cook_time       308
total_time       45
servings          0
yield           211
ingredients       0
directions        0
rating            0
url               0
cuisine_path      0
nutrition         0
timing            0
img_src           0
dtype: int64

In [4]:
nutrition_regex = re.compile(r"(?P<nutrition>[a-zA-Z\s]+)(?P<amount>\d+)(?P<unit>[a-z]+).*")
time_regex = re.compile(r"((?P<hours>\d+)\s+hrs)?\s*((?P<minutes>\d+)\s+mins)?\s*")


def parse_nutrition(nutrition_desc):
    result = {}
    for entry in nutrition_desc.split(", "):
        entry = entry.strip()
        matcher = nutrition_regex.match(entry)
        if not matcher:
            raise ValueError(nutrition_desc + " | " + entry)
        else:
            name = matcher.group("nutrition").strip()
            amount = float(matcher.group("amount"))
            unit = matcher.group("unit")
            result[name] = (amount, unit)
    return result


def parse_time(time_str):
    if not isinstance(time_str, str) and np.isnan(time_str):
        return np.NaN
    matcher = time_regex.match(time_str)
    if not matcher:
        raise ValueError(time_str)
    try:
        hours = float(matcher.group("hours"))
    except (IndexError, TypeError):
        hours = 0
    try:
        minutes = float(matcher.group("minutes"))
    except (IndexError, TypeError):
        minutes = 0
    return hours * 60 + minutes


def parse_path(path_str):
    return [p for p in path_str.split("/") if len(p.strip()) != 0]


recipe_df["nutrition"] = recipe_df.nutrition.apply(parse_nutrition)
recipe_df["prep_time"] = recipe_df.prep_time.apply(parse_time)
recipe_df["cook_time"] = recipe_df.cook_time.apply(parse_time)
recipe_df["total_time"] = recipe_df.total_time.apply(parse_time)
recipe_df["cuisine_path"] = recipe_df.cuisine_path.apply(parse_path)
recipe_df.head()

,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,"[Desserts, Fruit Desserts, Apple Dessert Recipes]","{'Total Fat': (18.0, 'g'), 'Saturated Fat': (7...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,Apple Pie by Grandma Ople,30.0,60.0,90.0,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,"[Desserts, Pies, Apple Pie Recipes]","{'Total Fat': (19.0, 'g'), 'Saturated Fat': (9...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
2,Sarah's Homemade Applesauce,10.0,15.0,25.0,4,NaN,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,"[Side Dish, Applesauce Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (3.0, 'mg'...","Prep Time: 10 mins, Cook Time: 15 mins, Total ...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...
3,Apple Crisp,30.0,45.0,75.0,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,"[Desserts, Crisps and Crumbles Recipes, Apple ...","{'Total Fat': (8.0, 'g'), 'Saturated Fat': (5....","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,Apple Pie Filling,20.0,20.0,160.0,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,"[Desserts, Pies, Apple Pie Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (61.0, 'mg...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...


In [5]:
recipe_reduced = recipe_df.drop(["yield", "url", "timing", "img_src"], axis=1)
recipe_reduced.head()

,recipe_name,prep_time,cook_time,total_time,servings,ingredients,directions,rating,cuisine_path,nutrition
0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,"[Desserts, Fruit Desserts, Apple Dessert Recipes]","{'Total Fat': (18.0, 'g'), 'Saturated Fat': (7..."
1,Apple Pie by Grandma Ople,30.0,60.0,90.0,8,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,"[Desserts, Pies, Apple Pie Recipes]","{'Total Fat': (19.0, 'g'), 'Saturated Fat': (9..."
2,Sarah's Homemade Applesauce,10.0,15.0,25.0,4,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,"[Side Dish, Applesauce Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (3.0, 'mg'..."
3,Apple Crisp,30.0,45.0,75.0,12,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,"[Desserts, Crisps and Crumbles Recipes, Apple ...","{'Total Fat': (8.0, 'g'), 'Saturated Fat': (5...."
4,Apple Pie Filling,20.0,20.0,160.0,40,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,"[Desserts, Pies, Apple Pie Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (61.0, 'mg..."


In [6]:
recipe_reduced.loc[recipe_reduced.prep_time.isna() &
                   ~recipe_reduced.cook_time.isna() &
                   ~recipe_reduced.total_time.isna(), "prep_time"] = recipe_reduced.total_time - \
                                                                     recipe_reduced.cook_time
recipe_reduced.loc[recipe_reduced.cook_time.isna() &
                   ~recipe_reduced.prep_time.isna() &
                   ~recipe_reduced.total_time.isna(), "cook_time"] = recipe_reduced.total_time - \
                                                                     recipe_reduced.cook_time
recipe_reduced.loc[recipe_reduced.total_time.isna() &
                   ~recipe_reduced.prep_time.isna() &
                   ~recipe_reduced.cook_time.isna(), "total_time"] = recipe_reduced.prep_time + \
                                                                     recipe_reduced.cook_time
recipe_reduced = recipe_reduced.loc[~recipe_reduced.prep_time.isna()].copy()
recipe_reduced.isna().sum()

recipe_name       0
prep_time         0
cook_time       263
total_time        0
servings          0
ingredients       0
directions        0
rating            0
cuisine_path      0
nutrition         0
dtype: int64

In [7]:
recipe_reduced.loc[recipe_reduced.cook_time.isna()].sample(10, random_state=SEED)

,recipe_name,prep_time,cook_time,total_time,servings,ingredients,directions,rating,cuisine_path,nutrition
493,Ponche de Frutas (Fruit Punch),20.0,NaN,20.0,20,"3 oranges, juiced, 3 lemons, juiced, ½ pound...","In a large punch bowl, combine orange juice, l...",4.6,"[Cuisine, Latin American, Mexican]","{'Total Fat': (0.0, 'g'), 'Sodium': (2.0, 'mg'..."
183,Vanilla Cherry Ice Cream,195.0,NaN,315.0,8,"2 cups heavy cream, 1 cup milk, ¾ cup white su...","Combine the cream, milk, and sugar in a bowl. ...",4.6,"[Desserts, Fruit Desserts, Cherry Dessert Reci...","{'Total Fat': (23.0, 'g'), 'Saturated Fat': (1..."
936,Summer Fruit Salad II,25.0,NaN,85.0,12,"3 cups cubed (1-inch) seedless watermelon, 1 l...","Gently toss the watermelon, peach, nectarine, ...",4.7,"[Salad, Fruit Salad Recipes, Blueberry Salad R...","{'Total Fat': (2.0, 'g'), 'Saturated Fat': (2...."
649,Caribbean Health Drink,10.0,NaN,10.0,2,"1 cup chopped carrot, 1 banana, 1 kiwi, peel...","Blend the carrot, banana, kiwi, apple, pineapp...",3.9,"[Cuisine, Latin American, Caribbean]","{'Total Fat': (1.0, 'g'), 'Saturated Fat': (0...."
780,Fruit Batido,10.0,NaN,10.0,6,"1 (12 fluid ounce) can evaporated milk, 1 cup ...","Blend milk, papaya, sugar, vanilla extract, an...",4.0,"[Drinks Recipes, Smoothie Recipes]","{'Total Fat': (5.0, 'g'), 'Saturated Fat': (3...."
295,Easy Watermelon Martini,10.0,NaN,10.0,1,"1 slice watermelon, rind removed, ½ cup ice cu...",Place watermelon in a cocktail shaker and crus...,4.4,"[Drinks Recipes, Cocktail Recipes, Vodka Drink...","{'Total Fat': (0.0, 'g'), 'Sodium': (3.0, 'mg'..."
680,Creamy Mango Smoothie,5.0,NaN,5.0,2,"¾ cup cold milk, ¼ cup vanilla yogurt, ¾ teasp...","Blend the milk, yogurt, vanilla extract, mango...",4.4,"[Drinks Recipes, Smoothie Recipes, Mango]","{'Total Fat': (3.0, 'g'), 'Saturated Fat': (2...."
766,White Peachy Sangria,30.0,NaN,510.0,20,"4 (750 milliliter) bottles moscato wine, 4 (12...","Stir moscato wine, lemon-lime soda, brandy, pe...",4.9,"[Drinks Recipes, Sangria Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (24.0, 'mg..."
246,Simple Cantaloupe Smoothie,10.0,NaN,10.0,1,1 cup vanilla yogurt (such as Dannon® Light an...,"Blend yogurt, orange juice, cantaloupe, and ic...",4.7,"[Drinks Recipes, Smoothie Recipes, Orange]","{'Total Fat': (4.0, 'g'), 'Saturated Fat': (2...."
248,Watermelon Ice Pops,10.0,NaN,130.0,6,"1 ½ cups watermelon, seeded and diced, ½ cup w...","Blend watermelon, water, honey, lemon juice, a...",4.7,"[Desserts, Frozen Dessert Recipes, Popsicle Re...","{'Total Fat': (0.0, 'g'), 'Sodium': (1.0, 'mg'..."


In [8]:
recipe_reduced["cook_time"] = recipe_reduced.cook_time.fillna(0)
recipe_reduced.loc[recipe_reduced.total_time < (recipe_reduced.prep_time +
                                                recipe_reduced.cook_time), "total_time"] = recipe_reduced.cook_time + recipe_reduced.prep_time
recipe_reduced.head()

,recipe_name,prep_time,cook_time,total_time,servings,ingredients,directions,rating,cuisine_path,nutrition
1,Apple Pie by Grandma Ople,30.0,60.0,90.0,8,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,"[Desserts, Pies, Apple Pie Recipes]","{'Total Fat': (19.0, 'g'), 'Saturated Fat': (9..."
2,Sarah's Homemade Applesauce,10.0,15.0,25.0,4,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,"[Side Dish, Applesauce Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (3.0, 'mg'..."
3,Apple Crisp,30.0,45.0,75.0,12,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,"[Desserts, Crisps and Crumbles Recipes, Apple ...","{'Total Fat': (8.0, 'g'), 'Saturated Fat': (5...."
4,Apple Pie Filling,20.0,20.0,160.0,40,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,"[Desserts, Pies, Apple Pie Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (61.0, 'mg..."
5,Easy Apple Crisp with Oat Topping,20.0,40.0,60.0,4,"6 apples - peeled, cored, and sliced, 2 table...",Preheat the oven to 350 degrees F (175 degrees...,4.7,"[Desserts, Crisps and Crumbles Recipes, Apple ...","{'Total Fat': (25.0, 'g'), 'Saturated Fat': (1..."


In [9]:
len(recipe_reduced.loc[recipe_reduced.prep_time +
                   recipe_reduced.cook_time !=
                   recipe_reduced.total_time].index)

370

In [10]:
recipe_reduced.loc[recipe_reduced.prep_time +
                   recipe_reduced.cook_time !=
                   recipe_reduced.total_time].sample(10, random_state=SEED)

,recipe_name,prep_time,cook_time,total_time,servings,ingredients,directions,rating,cuisine_path,nutrition
90,Apple Strudel Muffins,20.0,20.0,45.0,12,"2 cups all-purpose flour, 1 teaspoon baking po...",Preheat the oven to 375 degrees F (190 degrees...,4.5,"[Bread, Quick Bread Recipes, Muffin Recipes, A...","{'Total Fat': (10.0, 'g'), 'Saturated Fat': (6..."
387,Single Crust Peach Pie,30.0,0.0,80.0,8,"¾ cup white sugar, 2 tablespoons butter, softe...",Cream sugar and butter or margarine together. ...,4.4,"[Desserts, Pies, Fruit Pies, Peach Pie Recipes]","{'Total Fat': (10.0, 'g'), 'Saturated Fat': (4..."
522,Salmon with Pomegranate Glaze,10.0,11.0,36.0,4,"⅓ cup pomegranate juice, 1 tablespoon soy sauc...","Whisk pomegranate juice, soy sauce, and garlic...",4.0,"[Main Dishes, Seafood Main Dishes, Salmon, Sal...","{'Total Fat': (23.0, 'g'), 'Saturated Fat': (4..."
188,Almond Tortoni,35.0,0.0,515.0,8,"1 egg white, ¼ cup confectioners' sugar, 1 cu...","In a small bowl, beat egg white until foamy. G...",4.5,"[Desserts, Nut Dessert Recipes, Almond Dessert...","{'Total Fat': (19.0, 'g'), 'Saturated Fat': (7..."
480,Potato Plum Dumplings,35.0,20.0,175.0,12,"4 large russet potatoes, ½ teaspoon salt, 1 ta...","Scrub potatoes, and place them into a large po...",4.6,"[Cuisine, European, Austrian]","{'Total Fat': (10.0, 'g'), 'Saturated Fat': (6..."
981,Flan de Coco (Coconut Flan),10.0,60.0,550.0,12,"1 ½ cups white sugar, 1 (14 ounce) can sweeten...",Preheat oven to 350 degrees F (175 degrees C)....,4.7,"[Desserts, Custards and Puddings, Flan Recipes]","{'Total Fat': (9.0, 'g'), 'Saturated Fat': (5...."
1089,Chef John's Chocolate Energy Bars,15.0,0.0,180.0,12,"2 cups pitted Medjool dates, roughly chopped, ...","Place cashews, almonds, coconut, dates, cocoa,...",4.8,"[Appetizers and Snacks, Snacks, Granola Bar Re...","{'Total Fat': (22.0, 'g'), 'Saturated Fat': (7..."
602,Southern Style Banana Split Cake,30.0,0.0,120.0,12,"2 cups graham cracker crumbs, ¾ cup white suga...","Combine graham cracker crumbs, white sugar, an...",4.8,"[Desserts, Fruit Desserts, Banana Dessert Reci...","{'Total Fat': (46.0, 'g'), 'Saturated Fat': (2..."
1065,Coconut Date Balls,10.0,5.0,25.0,24,"2 eggs, beaten, 1 cup white sugar, 1 cup chop...","In a medium saucepan over medium heat, combine...",4.6,"[Desserts, Cookies, No-Bake Cookie Recipes]","{'Total Fat': (2.0, 'g'), 'Saturated Fat': (1...."
633,Simple Banana Bread,15.0,55.0,80.0,10,"cooking spray, 2 cups all-purpose flour, 1 t...",Preheat the oven to 350 degrees F (175 degrees...,4.0,"[Breakfast and Brunch, Breakfast Bread Recipes]","{'Total Fat': (5.0, 'g'), 'Saturated Fat': (1...."


In [11]:
recipe_reduced["auto_time"] = recipe_reduced.total_time - \
                              recipe_reduced.prep_time - \
                              recipe_reduced.cook_time
recipe_reduced.head()

,recipe_name,prep_time,cook_time,total_time,servings,ingredients,directions,rating,cuisine_path,nutrition,auto_time
1,Apple Pie by Grandma Ople,30.0,60.0,90.0,8,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,"[Desserts, Pies, Apple Pie Recipes]","{'Total Fat': (19.0, 'g'), 'Saturated Fat': (9...",0.0
2,Sarah's Homemade Applesauce,10.0,15.0,25.0,4,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,"[Side Dish, Applesauce Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (3.0, 'mg'...",0.0
3,Apple Crisp,30.0,45.0,75.0,12,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,"[Desserts, Crisps and Crumbles Recipes, Apple ...","{'Total Fat': (8.0, 'g'), 'Saturated Fat': (5....",0.0
4,Apple Pie Filling,20.0,20.0,160.0,40,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,"[Desserts, Pies, Apple Pie Recipes]","{'Total Fat': (0.0, 'g'), 'Sodium': (61.0, 'mg...",120.0
5,Easy Apple Crisp with Oat Topping,20.0,40.0,60.0,4,"6 apples - peeled, cored, and sliced, 2 table...",Preheat the oven to 350 degrees F (175 degrees...,4.7,"[Desserts, Crisps and Crumbles Recipes, Apple ...","{'Total Fat': (25.0, 'g'), 'Saturated Fat': (1...",0.0


In [12]:
recipe_reduced.to_json("../data/recipes.json", orient="records")